In [122]:
# Reactive programming layer for Julia
using Rocket 
# Core package for Constrained Bethe Free Energy minimsation with Factor graphs and message passing
using ReactiveMP 
# High-level user friendly probabilistic model and constraints specification language for ReactiveMP
using GraphPPL
# Optionally include Distributions.jl and Random from Base
using Distributions, Random

## General syntax for model creation

We use `@model` macro from `GraphPPL.jl` package to create a probabilistic model $p(s|y)$ and also specify extra constraints on variational family of distributions $\mathcal{Q}$.
Below there is a simple example of general syntax for model creation. In this tutorial we do not cover all possible way to create models or extra features of `GraphPPL.jl` and we refer a reader to the documentation for more rigorous explanations and examples.

In [35]:
# `@model` macro accepts a regular Julia function
@model function test_model1(s_mean, s_precision)
    
    # We use `randomvar` function to create 
    # random variables in our model
    s = randomvar()
    
    # `tilde` expression creates a functional dependencies
    # between variables in our model and can be read as 
    # `sampled from`
    s ~ GaussianMeanPrecision(s_mean, s_precision)
    
    # We use `datavar` function to create 
    # observed data variables in our models
    # We also need to specify the type of our data 
    # In this example it is `Float64`
    y = datavar(Float64)
    
    y ~ GaussianMeanPrecision(s, 1.0)
    
    return s, y
end

test_model1 (generic function with 3 methods)

`@model` macro creates a function with the same name and with the same set of input arguments as the original function (`test_model1(s_mean, s_precision)` in this example). However, the return value is modified in such a way to contain a reference to the model object as first value and user specified variables in a form of a tuple as second value.

In [87]:
model, (s, y) = test_model1(0.0, 1.0);

Later on we can examine our model structure with the help of some utility functions such as: 
- `getnodes()`: returns an array of factor nodes in a correposning factor graph
- `getrandom()`: returns an array of random variable in the model
- `getdata()`: returns an array of data inputs in the model
- `getconstant()`: return an array of constant values in the model

In [88]:
getnodes(model)

2-element Vector{ReactiveMP.AbstractFactorNode}:
 FactorNode:
 form            : NormalMeanPrecision
 sdtype          : Stochastic()
 interfaces      : (Interface(out, Marginalisation()), Interface(μ, Marginalisation()), Interface(τ, Marginalisation()))
 factorisation   : ((1, 2, 3),)
 local marginals : (:out_μ_τ,)
 metadata        : nothing
 pipeline        : FactorNodePipeline(functional_dependencies = DefaultFunctionalDependencies(), extra_stages = EmptyPipelineStage()

 FactorNode:
 form            : NormalMeanPrecision
 sdtype          : Stochastic()
 interfaces      : (Interface(out, Marginalisation()), Interface(μ, Marginalisation()), Interface(τ, Marginalisation()))
 factorisation   : ((1, 2, 3),)
 local marginals : (:out_μ_τ,)
 metadata        : nothing
 pipeline        : FactorNodePipeline(functional_dependencies = DefaultFunctionalDependencies(), extra_stages = EmptyPipelineStage()


In [38]:
getrandom(model) .|> name

1-element Vector{Symbol}:
 :s

In [39]:
getdata(model) .|> name

1-element Vector{Symbol}:
 :y

In [40]:
getconstant(model) .|> getconst

3-element Vector{Float64}:
 0.0
 1.0
 1.0

It is also possible to control flow statements in model specification such as `if` or `for` blocks

In [41]:
@model function test_model2(n)
    
    if n <= 1
        error("`n` argument must be greater than one.")
    end
    
    # `randomvar(n)` creates a dense sequence of 
    # random variables
    s = randomvar(n)
    
    # `datavar(Float64, n)` creates a dense sequence of 
    # observed data variables of type `Float64`
    y = datavar(Float64, n)
    
    s[1] ~ GaussianMeanPrecision(0.0, 0.1)
    y[1] ~ GaussianMeanPrecision(s[1], 1.0)
    
    for i in 2:n
        s[i] ~ GaussianMeanPrecision(s[i - 1], 1.0)
        y[i] ~ GaussianMeanPrecision(s[i], 1.0)
    end
    
    return s, y
end

test_model2 (generic function with 1 method)

In [44]:
model, (s, y) = test_model2(10);

In [53]:
# An amount of factor nodes in generated Factor Graph
getnodes(model) |> length

20

In [54]:
# An amount of random variables
getrandom(model) |> length

10

In [55]:
# An amount of data inputs
getdata(model) |> length

10

In [56]:
# An amount of constant values
getconstant(model) |> length

21

It is also possible to use complex expression inside functional dependencies expressions

```julia
y ~ NormalMeanPrecision(2.0 * (s + 1.0), 1.0)
```

`~` operator automatically creates a random variable if none was created before with the same name and errors if this name already exists

```julia
# s = randomvar() here is optional
# `~` creates random variables automatically
s ~ NormalMeanPrecision(0.0, 1.0)
```

An error example:

In [84]:
@model function error_model1()
    s = 1.0
    s ~ NormalMeanPrecision(0.0, 1.0)
end

LoadError: LoadError: Invalid name 's' for new random variable. 's' was already initialized with '=' operator before.
in expression starting at /Users/bvdmitri/.julia/dev/GraphPPL/src/GraphPPL.jl:161

By default `GraphPPL.jl` creates a new references for constants (literals like `0.0` or `1.0`) in a model. In some situtations it maybe not efficient especially if this constants represent some matrices. `GraphPPL.jl` will create a new copy of some constant matrix in a model every time it uses it. However it is possible to use `constavar()` function to create and reuse constant in the model specification syntax

```julia
# Creates constant reference in a model with a prespecified value
c = constvar(0.0)
```

An example:

In [86]:
@model function test_model5(dim::Int, n::Int, A::Matrix, P::Matrix, Q::Matrix)
    
    s = randomvar(n)
    
    y = datavar(Vector{Float64}, n)
    
    # Here we create constant references
    # for constant matrices in our model 
    # to make inference a little bit more efficient
    cA = constvar(A)
    cP = constvar(P)
    cQ = constvar(Q)
    
    s[1] ~ MvGaussianMeanCovariance(zeros(dim), cP)
    y[1] ~ MvGaussianMeanCovariance(s[1], cQ)
    
    for i in 2:n
        s[i] ~ MvGaussianMeanCovariance(cA * s[i - 1], cP)
        y[i] ~ MvGaussianMeanCovariance(s[i], cQ)
    end
    
    return s, y
end

test_model5 (generic function with 1 method)

`~` expression also may return a reference to a newly created node in a corresponding factor graph for better convenience or later usage:

```julia
@model function test_model()

    # In this example `ynode` refers to the corresponding 
    # `GaussianMeanVariance` node created in the factor graph
    ynode, y ~ GaussianMeanVariance(0.0, 1.0)
    
    return ynode, y
end
```

## Inference in ReactiveMP.jl

ReactiveMP.jl uses `Rocket.jl` library API for inference routines. `Rocket.jl` is a reactive programming extenstions library for Julia that is higly inspired by `RxJS` and similar libraries from `Rx` ecosystem. It consists of **observables**, **actors**, **subscriptions** and **operators**.

### Observables
Observables deliver their values over time

In [94]:
observable = timer(1000, 1000)

TimerObservable(1000, 1000)

Subscription allows to subscribe on future values in observable and actors specify what to do with new values:

In [106]:
actor = (value) -> println(value)
subscription1 = subscribe!(observable, actor)

TimerSubscription()

0
1
2
3
4


In [107]:
# We always need to unsubscribe from some observables
unsubscribe!(subscription1)

In [114]:
# We can modify our observables
modified = observable |> filter(d -> rem(d, 2) === 1) |> map(Int, d -> d ^ 2)

ProxyObservable(Int64, MapProxy(Int64))

In [115]:
subscription2 = subscribe!(modified, (value) -> println(value))

TimerSubscription()

1
9
25
49
81


In [116]:
unsubscribe!(subscription2)

`ReactiveMP.jl` library returns posterior marginals in a form of an observable. It is not possible to directly ask for a posterior marginal, but only to subscribe on its future updates. To get a reference for the posterior marginal of some random variable in a model `ReactiveMP.jl` exports two functions: 
- `getmarginal(x)`: for a single random variable `x`
- `getmarginals(xs)`: for a dense sequence of random variables `sx`

Lets see how it works in practice:

In [131]:
@model function coin_toss_model(n)

    # `datavar` creates data 'inputs' in our model
    # We will pass data later on to these inputs
    # In this example we create a sequence of inputs that accepts Float64
    y = datavar(Float64, n)
    
    # We endow θ parameter of our model with some prior
    θ ~ Beta(2.0, 7.0)
    
    # We assume that outcome of each coin flip 
    # is governed by the Bernoulli distribution
    for i in 1:n
        y[i] ~ Bernoulli(θ)
    end
    
    # We return references to our data inputs and θ parameter
    # We will use these references later on during inference step
    return y, θ
end

coin_toss_model (generic function with 1 method)

In [150]:
ctmodel, (y, θ) = coin_toss_model(500);

In [151]:
# As soon as we have a new value for marginal posterior over `θ` variable
# we simply print it
θ_subscription = subscribe!(getmarginal(θ), (marginal) -> println("New update: mean(θ) = ", mean(marginal), ", std(θ) = ", std(marginal)));

Next, lets define our dataset:

In [152]:
p = 0.75 # Bias of a coin

distribution = Bernoulli(p) 
dataset      = float.(rand(Bernoulli(p), 500));

To pass data to our model we use `update!` function

In [153]:
update!(y, dataset)

New update: mean(θ) = 0.756385068762279, std(θ) = 0.019008083755868727


In [154]:
# It is necessary to always unsubscribe from running observables
unsubscribe!(θ_subscription)

## Variational inference

### Factorisation constraints for variational family of distributions $\mathcal{Q}$

On a very high-level, ReactiveMP.jl is aimed to solve the Constrained Bethe Free Energy minimisation problem. For this task we often need to specify extra factorisation on variatonal family of distributions $q \in \mathcal{Q}$. For this purpose `@model` macro supports optional `where { ... }` clauses for `~` expressions in a model specification.

In [90]:
@model function test_model6(n)
    τ ~ GammaShapeRate(1.0, 1.0) 
    μ ~ NormalMeanVariance(0.0, 100.0)
    
    y = datavar(Float64, n)
    
    for i in 1:n
        # Here we assume a mean-field assumption on our 
        # variational family of distributions locally for the current node
        y[i] ~ NormalMeanPrecision(μ, τ) where { q = q(y[i])q(μ)q(τ) }
    end
    
    return μ, τ, y
end

test_model6 (generic function with 1 method)

There are several options to specify the mean-field factorisation constraint. 

```julia
y[i] ~ NormalMeanPrecision(μ, τ) where { q = q(y[i])q(μ)q(τ) } # With names from model specification
y[i] ~ NormalMeanPrecision(μ, τ) where { q = q(out)q(mean)q(precision) } # With names from node specification
y[i] ~ NormalMeanPrecision(μ, τ) where { q = MeanField() } # With alias name
```

It is also possible to use local structured factorisation:

```julia
y[i] ~ NormalMeanPrecision(μ, τ) where { q = q(y[i], μ)q(τ) } # With names from model specification
y[i] ~ NormalMeanPrecision(μ, τ) where { q = q(out, mean)q(precision) } # With names from node specification
```

An an option `@model` macro accepts optional arguments for model specification, one of which is `default_factorisation` that accepts `MeanField()` as its argument for better convenience

```julia
@model [ default_factorisation = MeanField() ] function test_model(...)
    ...
end
```

### Meta specification

During model specification some functional dependencies may accept an optional `meta` object in `where { ... }` clause. The purpose of the `meta` object is to adjust, modify or supply some extra information to the inference backend during messages computations. `meta` object for example may contain an approximation method that needs to be used during various approximations or it may specify the tradeoff between accuracy and performance:

```julia
# In this example `meta` object for autoregressive `AR` node specifes the variate type of 
# the autoregressive process and its order. In addition it specifies that messages computation rules 
# respect accuracy over speed with `ARsafe()` strategy. In contrast, `ARunsafe()` strategy tries to speedup computations
# by cost of possible numerical instabilities during an inference procedure
s[i] ~ AR(s[i - 1], θ, γ) where { q = q(s[i - 1], s[i])q(θ)q(γ), meta = ARMeta(Multivariate, order, ARsafe()) }
...
s[i] ~ AR(s[i - 1], θ, γ) where { q = q(s[i - 1], s[i])q(θ)q(γ), meta = ARMeta(Univariate, order, ARunsafe()) }
```

Another example with `GaussianControlledVariance`, or simply `GCV` [see Hierarchical Gaussian Filter], node:

```julia
# In this example we specify structured factorisation and flag meta with `GaussHermiteCubature` 
# method with `21` sigma points for approximation of non-lineariety between hierarchy layers
xt ~ GCV(xt_min, zt, real_k, real_w) where { q = q(xt, xt_min)q(zt)q(κ)q(ω), meta = GCVMetadata(GaussHermiteCubature(21)) }
```

Meta object is usefull to pass any extra information to a node that is not a random variable or constant model variable.

## Creating custom nodes and message computation rules

### Custom nodes

To create a custom functional form and to make it available during model specification `ReactiveMP.jl` exports the `@node` macro:

```julia
# `@node` macro accepts a name of the functional form, its type, either `Stochastic` or `Deterministic` and an array of interfaces:
@node NormalMeanVariance Stochastic [ out, μ, v ]

# Interfaces may have aliases for their names that might be convenient for factorisation constraints specification
@node NormalMeanVariance Stochastic [ out, (μ, aliases = [ mean ]), (v, aliases = [ var ]) ]

# `NormalMeanVariance` structure declaration must exist, otherwise `@node` macro will throw an error
struct NormalMeanVariance end 

@node NormalMeanVariance Stochastic [ out, μ, v ]

# It is also possible to use function objects as a node functional form
function dot end

# Syntax for functions is a bit differet, as it is necesssary to use `typeof(...)` function for them 
# out = dot(x, a)
@node typeof(dot) Deterministic [ out, x, a ]
```

**Note**: Deterministic nodes do not support factorisation constraints with `where { q = ... }` clause.

After that it is possible to use newly during model specification:

```julia
@model function test_model()
    ...
    y ~ dot(x, a)
    ...
end
```

### Custom messages computation rules

`ReactiveMP.jl` exports `@rule` macro to create custom messages computation rules. For example let us create a simple `+` node to be available for usage in the model specification usage. We refer to *A Factor Graph Approach to Signal Modelling , System Identification and Filtering* [ Sascha Korl, 2005, page 32 ] for a rigorous explanation `+` node in factor graphs. According to Korl, assuming that inputs are Gaussian Sum-Product message computation rule for `+` node is the following:

$$
\mu_z = \mu_x + \mu_y \\
V_z = V_x + V_y
$$

To specify this in `ReactiveMP.jl` we use `@node` and `@rule` macros:
 
```julia
@node typeof(+) Deterministic  [ z, x, y ]

@rule typeof(+)(:z, Marginalisation) (m_x::UnivariateNormalDistributionsFamily, m_y::UnivariateNormalDistributionsFamily) = begin
    mx, vx = mean_var(m_x)
    my, vy = mean_var(m_y)
    return NormalMeanVariance(mx + my, vx + vy)
end
```

In this example, for the `@rule` macro, we specify a type of our functional form: `typeof(+)`. Next, we specify an edge we are going to compute an outbound message for. `Marginalisation` indicates that the corresponding message respects the marginalisation constraint for posterior over corresponding edge:

$$
q(z) = \int q(z, x, y) \mathrm{d}x\mathrm{d}y
$$